# **Dependancies**

In [263]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras import *
from keras.callbacks import *
import os
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from commons import mean_absolute_percentage_error
from keras.layers import *
from sklearn.pipeline import Pipeline
import tensorflow as tf
from tensorflow.keras.losses import LogCosh

# **Loading Data**

In [264]:
PATH_TO_DATA = "reg_interval1.csv"
data = pd.read_csv(PATH_TO_DATA)

In [265]:
data.head()

,difficulty3var,fee_to_rewardUSD,hashrate90std,mediantransactionvalue90trxUSD,mining_profitability30trx,price14wmaUSD,price30emaUSD,price3wmaUSD,price7smaUSD,price7wmaUSD,price90momUSD,top100cap90sma,transactionvalue30smaUSD,transactionvalue90emaUSD,priceUSD
0,-4.294967e+09,24.447,5.438008e+18,1.707,0.516,13714,14124,11780,13261,12798,5091.0,16.875,96961,85113,10747
1,-5.368709e+09,23.309,5.589062e+18,1.716,0.446,13332,13959,11405,12933,12373,5773.0,16.886,96445,85012,11561
2,-5.368709e+09,19.255,5.634595e+18,1.722,0.376,12982,13804,11425,12605,12030,5483.0,16.897,96146,85180,11560
3,-5.368709e+09,13.105,6.187523e+18,1.715,0.162,12322,13456,11694,11695,11707,5555.0,16.941,94913,84139,11217
4,-5.368709e+09,11.452,6.338603e+18,1.705,0.090,12038,13285,11136,11487,11484,5254.0,16.958,94620,83633,10800


In [266]:
data.shape

(866, 15)

In [267]:
X = data

In [268]:
#X1 = data
#X1['target'] = X1['priceUSD'].shift(-1)


In [269]:
#X1.dropna(subset=['target'], inplace=True)

In [270]:
#X = data.iloc[:,:-1]

In [271]:
#y=data.iloc[:,-1:]

In [272]:
y = data['priceUSD'].shift(-1,fill_value=1)

In [273]:
#X1

In [274]:
y

0      11561
1      11560
2      11217
3      10800
4      11094
       ...  
861    11370
862    11348
863    11433
864    11582
865        1
Name: priceUSD, Length: 866, dtype: int64

In [275]:
def create_dataset(dataset, time_step=1):
    dataX = []
    datay = []
    for i in range(len(dataset) - time_step):
        a = dataset.iloc[i:(i + time_step), :].values  # Use .iloc for DataFrame
        dataX.append(a)
        datay.append(dataset.iloc[i + time_step, -1])
    return np.array(dataX), np.array(datay)


In [276]:
#time_step = 15
#X2 ,y2 = create_dataset(X1, time_step)

#print("X: ", X2.shape)
#print("y: ", y2.shape)

In [277]:
#X2

In [278]:
#y2

In [279]:
#y2_reshaped = y2.reshape(-1, 1, 1)

In [280]:
#y2_reshaped

In [281]:
X_train, X_test, y_train, y_test =train_test_split(X,y, test_size=0.2, train_size=0.8, shuffle=False, random_state=8)

In [282]:
X_train.shape

(692, 15)

In [283]:
estimators=[]

In [284]:
estimators.append(['robust',RobustScaler()])

In [285]:
estimators.append(['mixmax',MinMaxScaler()])

In [286]:
scale=Pipeline(estimators,verbose=True)

In [287]:
X_train=scale.fit_transform(X_train)

[Pipeline] ............ (step 1 of 2) Processing robust, total=   0.0s
[Pipeline] ............ (step 2 of 2) Processing mixmax, total=   0.0s


In [288]:
X_test=scale.transform(X_test)

In [289]:
X_train=np.reshape(X_train,(X_train.shape[0],1,X_train.shape[1]))

In [290]:
X_test=np.reshape(X_test,(X_test.shape[0],1,X_test.shape[1]))

In [291]:
y_train=y_train.values

In [292]:
y_train=np.reshape(y_train, (y_train.shape[0],1,1))

In [293]:
y_test=y_test.values

In [294]:
y_test=np.reshape(y_test,(y_test.shape[0],1,1))

In [295]:
X_train.shape

(692, 1, 15)

In [296]:
X_train

array([[[3.31408817e-14, 1.00000000e+00, 0.00000000e+00, ...,
         1.00000000e+00, 9.93024796e-01, 8.66966742e-01]],

       [[3.23876798e-14, 9.52563568e-01, 8.61345104e-03, ...,
         9.93785828e-01, 9.91538518e-01, 9.68742186e-01]],

       [[3.23876798e-14, 7.83576490e-01, 1.12098892e-02, ...,
         9.90184980e-01, 9.94010742e-01, 9.68617154e-01]],

       ...,

       [[2.53075824e-13, 2.60108378e-02, 7.38479840e-01, ...,
         9.72951491e-02, 1.00183945e-01, 4.04351088e-01]],

       [[1.28981646e-01, 3.26385994e-02, 7.43398522e-01, ...,
         9.49828990e-02, 9.80501803e-02, 3.81345336e-01]],

       [[1.93684164e-01, 4.46019175e-02, 7.46143017e-01, ...,
         9.38508599e-02, 9.72113899e-02, 3.96724181e-01]]])

In [297]:
y_train.shape

(692, 1, 1)

In [298]:
y_train

array([[[11561]],

       [[11560]],

       [[11217]],

       [[10800]],

       [[11094]],

       [[11384]],

       [[11085]],

       [[11294]],

       [[11710]],

       [[11372]],

       [[10781]],

       [[10154]],

       [[ 9755]],

       [[ 8748]],

       [[ 9091]],

       [[ 8826]],

       [[ 7704]],

       [[ 6852]],

       [[ 7933]],

       [[ 8256]],

       [[ 8300]],

       [[ 8691]],

       [[ 8234]],

       [[ 8662]],

       [[ 8657]],

       [[ 9055]],

       [[ 9838]],

       [[10058]],

       [[10713]],

       [[10772]],

       [[10902]],

       [[11551]],

       [[10851]],

       [[10325]],

       [[10046]],

       [[ 9975]],

       [[ 9649]],

       [[ 9939]],

       [[10575]],

       [[10607]],

       [[10666]],

       [[11002]],

       [[11365]],

       [[11271]],

       [[11537]],

       [[11051]],

       [[10428]],

       [[ 9724]],

       [[ 8927]],

       [[ 9252]],

       [[ 9065]],

       [[ 9457]],

       [[ 91

In [299]:
#X_train, X_test, y_train, y_test =train_test_split(X2,y2_reshaped, test_size=0.2, train_size=0.8, shuffle=False, random_state=8)

In [300]:
#X_train.shape[2]

In [301]:
#y_train.shape

In [302]:
#X_test.shape

# **Model Architecture + Training**

In [303]:
def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

In [304]:

adam = optimizers.Adam(learning_rate=lr_schedule(0), amsgrad=True)

Learning rate:  0.001


In [305]:
model = Sequential()
model.add(Bidirectional(LSTM(350, return_sequences=True, activation='relu'), input_shape=(1, X_train.shape[2])))
model.add(Bidirectional(LSTM(350, return_sequences=True, activation='relu')))
model.add(Dense(1))
model.compile(loss="log_cosh", optimizer=adam, metrics=['mae'])

E:\DS440proj\BTCpred\.venv\lib\site-packages\keras\src\layers\core\wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [306]:
mcp_save = ModelCheckpoint('model/LSTM_reg_seven_new.keras', save_best_only=True, monitor='val_loss', mode='auto')
earlyStopping = EarlyStopping(monitor='val_loss', patience=100, verbose=1, mode='auto')

In [307]:
#model.compile(optimizer='adam', loss='log_cosh', metrics=['accuracy'])
model.fit(X_train,y_train, epochs=5000, batch_size=32, validation_data=(X_test,y_test), callbacks=[mcp_save,earlyStopping])

Epoch 1/5000
22/22 ━━━━━━━━━━━━━━━━━━━━ 7s 58ms/step - loss: 7808.2339 - mae: 7808.9272 - val_loss: 10005.9170 - val_mae: 10006.6104
Epoch 2/5000
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - loss: 7892.3115 - mae: 7893.0044 - val_loss: 9907.9727 - val_mae: 9908.6670
Epoch 3/5000
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - loss: 7892.7817 - mae: 7893.4751 - val_loss: 9457.4023 - val_mae: 9458.0957
Epoch 4/5000
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - loss: 7320.0801 - mae: 7320.7739 - val_loss: 8088.8145 - val_mae: 8089.5073
Epoch 5/5000
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - loss: 6165.1089 - mae: 6165.8027 - val_loss: 4848.3867 - val_mae: 4849.0796
Epoch 6/5000
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - loss: 3419.8320 - mae: 3420.5251 - val_loss: 1485.4719 - val_mae: 1486.1650
Epoch 7/5000
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 1253.5497 - mae: 1254.2429 - val_loss: 1551.4818 - val_mae: 1552.1749
Epoch 8/5000
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 1098.6567 - mae: 109

# **Testing Model**

In [308]:
#from tensorflow.keras.models import load_model
#prediction_model = load_model('model/LSTM_reg_seven_new.keras',compile=False)

In [309]:
y_pred = np.ravel(model.predict(X_test))

6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step


In [310]:
y_test=np.ravel(y_test)

In [311]:
y_pred

array([ 7323.6514,  7586.584 ,  7636.0786,  7680.34  ,  7784.344 ,
        7802.5576,  8238.201 ,  8892.764 ,  8846.62  ,  8948.016 ,
        9057.367 ,  8862.622 ,  8952.757 ,  9136.107 ,  9459.869 ,
        9925.289 ,  9770.205 ,  8797.326 ,  8707.792 ,  8686.37  ,
        8937.972 ,  9405.658 ,  9454.712 ,  9397.748 ,  9582.138 ,
        9666.364 ,  9608.123 ,  9643.986 ,  9293.45  ,  9080.916 ,
        9157.621 ,  9109.732 ,  8822.974 ,  8865.863 ,  9036.212 ,
        9319.383 ,  9477.028 ,  9567.911 ,  9616.686 ,  9615.891 ,
        9899.273 ,  9602.479 ,  9771.948 ,  9888.785 ,  9763.079 ,
        9718.87  ,  9786.732 ,  9786.277 ,  9876.376 ,  9779.065 ,
        9551.995 ,  9555.947 ,  9543.391 ,  9374.26  ,  9606.957 ,
        9741.993 ,  9650.619 ,  9450.519 ,  9434.457 ,  9475.685 ,
        9572.674 ,  9731.095 ,  9569.609 ,  9366.868 ,  9324.092 ,
        9249.74  ,  9195.352 ,  9241.484 ,  9277.82  ,  9307.481 ,
        9293.773 ,  9221.574 ,  9240.321 ,  9212.298 ,  9326.0

In [312]:
r2=r2_score(y_test,y_pred) #testing score/ r^2
r2

0.511872305058754

In [313]:
mae=mean_absolute_error(y_test,y_pred) #mae
mae

218.2836184446839

In [314]:
rmse=np.sqrt(mean_squared_error(y_test,y_pred)) #rmse
rmse

896.9759308024244

In [315]:
mape=mean_absolute_percentage_error(y_test,y_pred) #mape
mape

6601.5760569922295

In [316]:
pd.DataFrame(zip(['MAE','RMSE','MAPE','R^2'],[mae,rmse,mape,r2])).transpose()

,0,1,2,3
0,MAE,RMSE,MAPE,R^2
1,218.283618,896.975931,6601.576057,0.511872


In [317]:
pd.DataFrame([y_test,y_pred]).transpose()

,0,1
0,7519.0,7323.651367
1,7541.0,7586.583984
2,7583.0,7636.078613
3,7705.0,7680.339844
4,7726.0,7784.344238
...,...,...
169,11370.0,11347.274414
170,11348.0,11283.596680
171,11433.0,11245.944336
172,11582.0,11355.526367
